In [ ]:
import pandas as pd
import numpy as np



In [ ]:
df = pd.read_csv('/content/routes_contries_airports.csv')
df.info()

In [ ]:
# Replace commas with periods for decimal separators and handle whitespace
df['Latitude_Partida'] = df['Latitude_Partida'].str.replace(',', '.', regex=True).str.strip().astype(float, errors='coerce')
df['Latitude_Chegada'] = df['Latitude_Chegada'].str.replace(',', '.', regex=True).str.strip().astype(float, errors='coerce')
df['Longitude_Partida'] = df['Longitude_Partida'].str.replace(',', '.', regex=True).str.strip().astype(float, errors='coerce')
df['Longitude_Chegada'] = df['Longitude_Chegada'].str.replace(',', '.', regex=True).str.strip().astype(float, errors='coerce')

In [ ]:
# Função para calcular a distância entre dois pontos usando a fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    # Converter de graus para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Diferenças das coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Raio da Terra em quilômetros
    r = 6371
    return c * r

# Aplicar a função ao dataset carregado anteriormente
df['Distancia_km'] = df.apply(
    lambda row: haversine(row['Latitude_Partida'], row['Longitude_Partida'], row['Latitude_Chegada'], row['Longitude_Chegada']),
    axis=1
)

In [ ]:
df.to_csv('/content/routes_with_distance.csv',index=False)

In [ ]:
df

In [ ]:
df2 = pd.read_csv("/content/airportsNew.csv")
df2.info()

In [ ]:
df2["Latitude"] = df2["Latitude"].round(4)
df2["Longitude"] = df2["Longitude"].round(4)
df2

In [ ]:
df2.to_csv('/content/airportsNewTratado.csv',index=False)

In [ ]:
df_routes = pd.read_csv("/content/routes_with_distance.csv")


df_airports = pd.read_csv("/content/airportsNewTratado2.csv")

In [ ]:
df_routes.info()

In [ ]:
df_airports.info()

In [ ]:
# Renomear as colunas do DataFrame df_routes com nomes apropriados
""" df_routes.columns = [
    'CodigoCompanhiaAerea', 'NumeroVoo', 'AeroportoPartida', 'ID_AeroportoPartida',
    'AeroportoChegada', 'ID_AeroportoChegada', 'CodeShare', 'Paradas', 'Equipamento'
]"""

# Merge do DataFrame de rotas com o de aeroportos para Aeroporto de Partida
df_routes_merged_departure = df_routes.merge(
    df_airports[['Airport Name', 'Country', 'City', 'Latitude', 'Longitude']],
    left_on='AeroportoPartida',
    right_on='Airport Name',
    how='left'
).drop(columns=['Airport Name'])

# Renomear as colunas adicionadas para diferenciar Partida e Chegada
df_routes_merged_departure = df_routes_merged_departure.rename(columns={
    'City': 'Cidade_Partida',
    'Country': 'Pais_Partida',
    'Latitude': 'Latitude_Partida',
    'Longitude': 'Longitude_Partida'
})

# Merge do DataFrame resultante com o de aeroportos para Aeroporto de Chegada
df_routes_merged = df_routes_merged_departure.merge(
    df_airports[['Airport Name', 'Country','City', 'Latitude', 'Longitude']],
    left_on='AeroportoChegada',
    right_on='Airport Name',
    how='left'
).drop(columns=['Airport Name'])

# Renomear as colunas adicionadas para diferenciar Partida e Chegada
df_routes_merged = df_routes_merged.rename(columns={
    'City': 'Cidade_Chegada',
    'Country': 'Pais_Chegada',
    'Latitude': 'Latitude_Chegada',
    'Longitude': 'Longitude_Chegada'
})

# Salvar o DataFrame resultante em um novo arquivo CSV
output_path_corrected = 'routes_with_airports_corrected.csv'
df_routes_merged.to_csv(output_path_corrected, index=False)

In [ ]:
df_routes_merged